In [9]:
import re
import os
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from datetime import datetime
from PIL import Image

In [26]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Pratham
[nltk_data]     jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [59]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Pratham
[nltk_data]     jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:

# Download the NLTK tagger if necessary
nltk.download("averaged_perceptron_tagger")

# Define the list of keywords to extract
keyword_list = ["eyebrows", "eyes", "hair", "head", "lips", "mustache", "nose"]

# Define the path to the image folder
image_folder = "C:/Users/Pratham jain/Desktop/RIT/rit/semister 8/major project/speech-to-text/images"

# Define the path to the feedback file
feedback_file = "C:/Users/Pratham jain/Desktop/RIT/rit/semister 8/major project/speech-to-text/feedback.txt"

#adjectives list
adjective_list = list(wn.all_eng_synsets(wn.ADJ))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pratham jain\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [39]:
adjective_list

[Synset('able.a.01'),
 Synset('unable.a.01'),
 Synset('abaxial.a.01'),
 Synset('adaxial.a.01'),
 Synset('acroscopic.a.01'),
 Synset('basiscopic.a.01'),
 Synset('abducent.a.01'),
 Synset('adducent.a.01'),
 Synset('nascent.a.01'),
 Synset('emergent.s.02'),
 Synset('dissilient.s.01'),
 Synset('parturient.s.02'),
 Synset('dying.a.01'),
 Synset('moribund.s.02'),
 Synset('last.s.05'),
 Synset('abridged.a.01'),
 Synset('cut.s.03'),
 Synset('half-length.s.02'),
 Synset('potted.s.03'),
 Synset('unabridged.a.01'),
 Synset('full-length.s.02'),
 Synset('absolute.a.01'),
 Synset('direct.s.10'),
 Synset('implicit.s.02'),
 Synset('infinite.s.04'),
 Synset('living.s.03'),
 Synset('relative.a.01'),
 Synset('relational.s.01'),
 Synset('absorbent.a.01'),
 Synset('absorbefacient.s.01'),
 Synset('assimilating.s.01'),
 Synset('hygroscopic.s.01'),
 Synset('receptive.s.04'),
 Synset('shock-absorbent.s.01'),
 Synset('spongy.s.02'),
 Synset('thirsty.s.04'),
 Synset('nonabsorbent.a.01'),
 Synset('repellent.s.03'

In [74]:
# Open the "op.txt" file and read the latest line
with open("op.txt", "r") as file:
    line = file.readlines()[-1].strip()

# Define a regular expression to match adjectives
adj_regex = re.compile(r"\b(?:" + "|".join([adj.name() for adj in adjective_list]) + r")\b", re.IGNORECASE)

# Tokenize the sentence into words and tags
tagged_words = nltk.pos_tag(word_tokenize(line))

# Initialize empty lists for adjectives and keywords
adjectives = []
keywords = []

# Loop through each word and tag
for i, (word, tag) in enumerate(tagged_words):
    # Check if the word is a keyword
    if word.lower() in keyword_list:
        # Check if there is an adjective before this keyword
        if i > 0 and tagged_words[i-1][1] in ["JJ", "JJR", "JJS"]:
            # Add the adjective and keyword to their respective lists
            adjectives.append(tagged_words[i-1][0])
            keywords.append(word)


print(keywords)

#lets improve the code in more specific way, lets say if the sentence say "that beautiful girl has short hair and sharp eyes" the extracted words would be "short hair" and "sharp eyes" which need to be displayed but if in the eyes subfolder there is no "sharp eyes" image then the extracted word whose image is not found would be appended to the feedback.txt file along with the sentence it was extracted from with details such as date, there everytime a image is not found those extracted words will be stored in feedback.txt
#In this code, we first define the list of keywords to extract. We then download the NLTK tagger if necessary and tokenize the sentence into words and tags using the pos_tag function.

#Next, we initialize empty lists for adjectives and keywords. We then loop through each word and tag, and check if the word is a keyword. If the word is a keyword, we check if there is an adjective before this keyword. If there is, we add the adjective and keyword to their respective lists.

#After that, we combine the adjectives and keywords into phrases, just as in the previous code.

#Finally, we define the path to the image folder and loop through each phrase. For each phrase, we get the keyword from the phrase and define the path to the subfolder. We then search for the image file in the subfolder using the os.listdir function. If the image file is found, we combine the subfolder path and the image file name and display the image using the PIL library.

['hair', 'head', 'Eyes']


In [75]:
print(adjectives)

['short', 'kpt', 'tired']


In [76]:
# # Combine the adjectives and keywords into phrases
phrases = [" ".join(adj_keyword) for adj_keyword in zip(adjectives, keywords) if adj_keyword]
print(phrases)


['short hair', 'kpt head', 'tired Eyes']


In [77]:
# Loop through each phrase and display the corresponding image
for phrase in phrases:
    # Get the keyword from the phrase
    keyword = phrase.split()[-1]
    # Define the path to the subfolder
    subfolder = os.path.join(image_folder, keyword)
    # Search for the image file in the subfolder
    for file in os.listdir(subfolder):
        if phrase.lower() in file.lower():
            # Combine the subfolder path and the image file name
            image_path = os.path.join(subfolder, file)
            # Display the image
            Image.open(image_path).show()
            # Exit the loop since we found the image
            break
    else:
        # Image not found, append the phrase to the feedback file
        with open(feedback_file, "a") as file:
            file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {phrase} (image not found in {subfolder})\n")

